In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
import torch.nn.functional as F
from scipy.sparse import csgraph
from scipy.optimize import minimize
from sklearn.neighbors import NearestNeighbors
from sklearn.manifold import SpectralEmbedding
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import rbf_kernel
from scipy.sparse.csgraph import laplacian as csgraph_laplacian
from sklearn.metrics.pairwise import euclidean_distances
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigsh
from scipy.sparse import diags
from numpy.linalg import eig
from numpy.linalg import eigh
from scipy.linalg import block_diag
from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Define the CNN architecture with specified layers
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 100, kernel_size=5)
        self.conv2 = nn.Conv2d(100, 150, kernel_size=5)
        self.conv3 = nn.Conv2d(150, 200, kernel_size=3)
        self.mp = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(200 * 2 * 2, 300)  # Assuming the feature map size is 2x2
        self.fc2 = nn.Linear(300, 10)

    def forward(self, x):
        x = self.mp(F.relu(self.conv1(x)))
        x = self.mp(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = x.view(-1, 200 * 2 * 2)  # Flatten the tensor for the fully connected layer
        penultimate = F.relu(self.fc1(x))
        x = self.fc2(penultimate)
        return F.log_softmax(x, dim=1), penultimate

# Data preparation
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

indices = np.hstack([np.where(train_dataset.targets == i)[0][:100] for i in range(10)])
subset_train_dataset = Subset(train_dataset, indices)
train_loader = DataLoader(subset_train_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# Initialize and train 5 separate CNNs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models = [CustomCNN().to(device) for _ in range(5)]
criterion = nn.CrossEntropyLoss()

model_features = {i: [] for i in range(len(models))}

for i, model in enumerate(models):
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

    # Training loop
    for epoch in range(1, 101):  # 100 epochs
        model.train()
        model_features[i].clear()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output, penultimate = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            model_features[i].append(penultimate.detach().cpu().numpy())
        scheduler.step()  # Anneal the learning rate
        if epoch == 100:
            print(f'Model {i+1}, Epoch {epoch}, Loss: {loss.item()}')
    model_features[i] = np.concatenate(model_features[i], axis=0)

100%|██████████| 9912422/9912422 [00:00<00:00, 136722630.44it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 44274741.89it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 31839947.87it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 23665253.13it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Model 1, Epoch 100, Loss: 0.11837957054376602
Model 2, Epoch 100, Loss: 0.11553546041250229
Model 3, Epoch 100, Loss: 0.1151246577501297
Model 4, Epoch 100, Loss: 0.14135919511318207
Model 5, Epoch 100, Loss: 0.12834122776985168


In [3]:
def evaluate_model(model, test_loader, device):
    features, test_targets = [], []
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # No gradients required
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            outputs, penultimate = model(data)
            _, predicted = torch.max(outputs.data, 1)  # Get the index of the max log-probability
            total += target.size(0)
            correct += (predicted == target).sum().item()
            features.append(penultimate.cpu().numpy())
            test_targets.append(target.cpu().numpy())
            
    accuracy = correct / total
    features = np.concatenate(features, axis=0)
    test_targets = np.concatenate(test_targets, axis=0)
    return accuracy, features, test_targets

accuracies = []
multi_fea = []
for i, model in enumerate(models):
    accuracy, features, test_targets = evaluate_model(model, test_loader, device)
    accuracies.append(accuracy)
    multi_fea.append(features)
    print(f'Model {i+1} Accuracy: {accuracy * 100:.2f}%')

Model 1 Accuracy: 92.26%
Model 2 Accuracy: 92.68%
Model 3 Accuracy: 92.15%
Model 4 Accuracy: 92.06%
Model 5 Accuracy: 92.08%


In [4]:
# n_components = 10  # Number of desired dimensions
# svd_model = TruncatedSVD(n_components=n_components)
# svd_features_train = svd_model.fit_transform(np.concatenate(list(model_features.values()), axis=-1))

# # Train a classifier using the SVD-reduced training features
# classifier = LogisticRegression(max_iter=5000)
# classifier.fit(svd_features_train, train_dataset.targets[indices].numpy())

# # Function to evaluate the model using the Truncated SVD on test data
# def evaluate_with_svd(models, test_loader, classifier, device, svd_model):
#     test_features = []
#     test_targets = []
#     with torch.no_grad():
#         for data, target in test_loader:
#             data = data.to(device)
#             # Extract features from all models and concatenate them
#             concatenated_features = np.concatenate([model(data)[1].cpu().numpy() for model in models], axis=-1)
#             print(concatenated_features.shape)
#             test_features.append(concatenated_features)
#             test_targets.append(target.cpu().numpy())
    
#     # Apply Truncated SVD to the concatenated test features
#     test_features = np.concatenate(test_features, axis=0)
#     print(test_features.shape)
#     svd_test_features = svd_model.transform(test_features)  # Transform using the fitted SVD model
    
#     # Predict using the trained classifier
#     predictions = classifier.predict(svd_test_features)
#     test_targets = np.concatenate(test_targets, axis=0)
    
#     # Calculate accuracy
#     accuracy = np.mean(predictions == test_targets)
#     return accuracy

# # Evaluate the classifier with the test set
# test_accuracy = evaluate_with_svd(models, test_loader, classifier, device, svd_model)
# print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

(1000, 1500)
(1000, 1500)
(1000, 1500)
(1000, 1500)
(1000, 1500)
(1000, 1500)
(1000, 1500)
(1000, 1500)
(1000, 1500)
(1000, 1500)
(10000, 1500)
Test Accuracy: 91.62%


In [5]:
# Function to concatenate features from all models
def concatenate_features(models, data_loader, device):
    concatenated_features = []
    targets = []
    with torch.no_grad():
        for data, target in data_loader:
            data = data.to(device)
            # Extract and concatenate features from all models
            model_features = [model(data)[1].cpu().numpy() for model in models]
            concatenated_features.append(np.concatenate(model_features, axis=1))
            targets.append(target.cpu().numpy())
    return np.concatenate(concatenated_features), np.concatenate(targets)

# Concatenate training and test data features
train_features, train_targets = concatenate_features(models, train_loader, device)
test_features, test_targets = concatenate_features(models, test_loader, device)
combined_features = np.vstack((train_features, test_features))

# Apply spectral embedding to the combined set
embedding = SpectralEmbedding(n_components=10, affinity='nearest_neighbors', n_neighbors=5)
combined_embedded_features = embedding.fit_transform(combined_features)

# Split the embedded features back into training and test sets
train_embedded_features = combined_embedded_features[:len(train_features)]
test_embedded_features = combined_embedded_features[len(train_features):]

# Train a classifier using the embedded training features
classifier = LogisticRegression(max_iter=1000)
classifier.fit(train_embedded_features, train_targets)

# Predict using the trained classifier on the embedded test features
predictions = classifier.predict(test_embedded_features)

# Calculate accuracy
accuracy = np.mean(predictions == test_targets)
print(f'Accuracy of the model on the test set using Spectral Embedding: {accuracy * 100:.2f}%')


Accuracy of the model on the test set using Spectral Embedding: 94.02%


In [6]:
# def global_coordinate_alignment(part_embeddings, n_components, r):
#     # Number of views
#     m = len(part_embeddings)
#     # Total number of samples
#     n_samples = part_embeddings[0].shape[0]
    
#     # Initialize global embedding Y and weights alpha
#     Y_global_init = np.random.rand(n_samples, n_components)
#     alphas_init = np.full(m, 1 / m)

#     # Flatten the initial global embedding and concatenate with weights
#     x0 = np.concatenate([Y_global_init.flatten(), alphas_init], axis=0)

#     # Define the objective function
#     def objective(x):
#         # Extract Y and alphas from the flattened vector
#         Y = x[:-m].reshape(n_samples, n_components)
#         alphas = x[-m:]
        
#         # Compute the objective function value
#         loss = 0
#         for i, Y_i in enumerate(part_embeddings):
#             L_i = compute_graph_laplacian(Y_i)  
#             loss += alphas[i]**r * np.trace(np.dot(Y.T, np.dot(L_i, Y)))
#         return loss

#     # Define the constraint for alpha (they should sum to 1)
#     def constraint_alpha(x):
#         return np.sum(x[-m:]) - 1

#     # Define the constraints for the optimization
#     cons = ({'type': 'eq', 'fun': constraint_alpha})

#     # Perform the optimization
#     result = minimize(
#         fun=objective,
#         x0=x0,
#         constraints=cons,
#         method='SLSQP',  # Sequential Least Squares Programming
#         options={'disp': True}
#     )

#     # Extract the optimized Y and alphas from the result
#     Y_global_optimized = result.x[:-m].reshape(n_samples, n_components)
#     alphas_optimized = result.x[-m:]
    
#     return Y_global_optimized, normalize(alphas_optimized.reshape(1, -1), norm='l1').flatten()

# # This function needs to be implemented
# def compute_graph_laplacian(Y_i):
#     connectivity = kneighbors_graph(Y_i, n_neighbors=20, include_self=True)
#     laplacian_matrix = csgraph_laplacian(connectivity, normed=True)
#     return laplacian_matrix

# # Part embeddings from different views/models would be used as input for this function
# # part_embeddings = [...]
# # Assuming n_components and r are defined
# # Y_global, alphas = global_coordinate_alignment(part_embeddings, n_components, r)
# multi_features = []
# for i in range(5):
#     multi_features.append(model_features[i])
# global_coordinate_alignment(multi_features, 9, 1.5)

In [7]:
# import numpy as np
# from scipy.optimize import minimize
# from scipy.sparse import diags

# # Let's assume we have part_embeddings which is a list of numpy arrays,
# # each containing the part-based low-dimensional embeddings for different views.

# # Initialize the global embedding Y and the weights alpha for each view
# Y_global_init = np.random.rand(d, n)  # d is the target dimensionality, n is the number of data points
# alphas = np.full(len(part_embeddings), 1 / len(part_embeddings))  # Start with uniform weights

# # Define the objective function for global coordinate alignment
# def global_alignment_objective(Y_global, part_embeddings, alphas, r):
#     Y_global = Y_global.reshape(d, n)
#     objective = 0
#     for Y_part, alpha in zip(part_embeddings, alphas):
#         # Compute the weighted trace of Y_part's graph Laplacian and Y_global
#         L_part = compute_graph_laplacian(Y_part)  # You need to implement this based on the paper
#         objective += alpha**r * np.trace(Y_global.T @ L_part @ Y_global)
#     return objective

# # Define the constraint for the optimization: YY^T = I
# def orthogonality_constraint(Y_global):
#     Y_global = Y_global.reshape(d, n)
#     return np.dot(Y_global, Y_global.T) - np.eye(d)

# # Flatten Y_global for scipy's optimizer
# Y_global_flattened = Y_global_init.flatten()

# # Run the optimization
# result = minimize(
#     fun=global_alignment_objective,
#     x0=Y_global_flattened,
#     args=(part_embeddings, alphas, r),
#     constraints={'type': 'eq', 'fun': orthogonality_constraint},
#     method='SLSQP',  # Sequential Least Squares Programming
#     options={'disp': True}
# )

# # Reshape the result back to the matrix form
# Y_global_optimized = result.x.reshape(d, n)